In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [2]:
# data preparation
df_train = pd.read_csv('data/training.csv')
df_test = pd.read_csv('data/testing.csv')
df = pd.read_csv('data/race-result-horse.csv')
m, n = np.shape(df)
m_train, _ = np.shape(df_train)
m_test, _ = np.shape(df_test)

#produce dicts for horse,jockey,trainer
horse=list()
jockey=list()
trainer=list()
for i in range(len(df)):
    if df.horse_name[i] not in horse:
        horse.append(df.horse_name[i])
    if df.jockey[i] not in jockey:
        jockey.append(df.jockey[i])
    if df.trainer[i] not in jockey:
        trainer.append(df.trainer[i])
train_horse=np.zeros((m_train,1))
train_jockey=np.zeros((m_train,1))
train_trainer=np.zeros((m_train,1))

In [3]:
# prepare training data
for i in range(m_train):
    train_horse[i,0] = horse.index(df_train.horse_name[i])
    train_jockey[i,0] = jockey.index(df_train.jockey[i])
    train_trainer[i,0] = trainer.index(df_train.trainer[i])
actual_weight = df_train.actual_weight.reshape((m_train,1))
declared_weight = df_train.declared_horse_weight.reshape((m_train,1))
draw = df_train.draw.reshape((m_train,1))
win_odds = df_train.win_odds.reshape((m_train,1))
race_distance = df_train.race_distance.reshape((m_train,1))

# we use horse, jockey, trainer, actual weight, declared weight, win odds, race distance as independent variables
X_train = np.hstack((train_horse, train_jockey, train_trainer, actual_weight,
                     declared_weight, draw, win_odds, race_distance)) 
y_train = df_train.finishing_position

# prepare test data
test_horse=np.zeros((m_test,1))
test_jockey=np.zeros((m_test,1))
test_trainer=np.zeros((m_test,1))

#print(df_test.horse_name[])
for i in range(len(df_test)):
    test_horse[i,0] = horse.index(df_test.horse_name[i])
    test_jockey[i,0] = jockey.index(df_test.jockey[i])
    test_trainer[i,0] = trainer.index(df_test.trainer[i])
actual_weight_test = df_test.actual_weight.reshape((m_test,1))
declared_weight_test = df_test.declared_horse_weight.reshape((m_test,1))
draw_test = df_test.draw.reshape((m_test,1))
win_odds_test = df_test.win_odds.reshape((m_test,1))
race_distance_test = df_test.race_distance.reshape((m_test,1))
X_test = np.hstack((test_horse,test_jockey,test_trainer,actual_weight_test,
                    declared_weight_test, draw_test, win_odds_test, race_distance_test))
y_test = df_test.finishing_position

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # Remove the CWD from sys.path while we load

In [4]:
#3.1.1 Logistic 
lr_model = LogisticRegression()
lr_model.fit(X_train,y_train)
lr_result = lr_model.predict(X_test)
# print(lr_result.shape)
lr_score = lr_model.score(X_test,y_test)
print(lr_score)

0.13076792099438106


In [5]:
# 3.1.2 Naïve Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
nb_score = nb_model.score(X_test, y_test)
print(nb_score)


0.11901924059254214


In [6]:
# 3.1.3 SVM
svm_model = SVC(kernel='linear')
print('model')
svm_model.fit(X_train, y_train)
print('fit')
svm_score = svm_model.score(X_test, y_test)
print(svm_score)

model


KeyboardInterrupt: 